In [46]:
import os, sys
sys.path.insert(0, os.path.dirname(os.getcwd())) 

In [75]:
import requests
import time
from typing import Dict, Any
import pandas as pd
import numpy as np
from functools import reduce

In [33]:
ticker = 'AAPL'

## Base params

In [40]:
url = 'https://query{query_id}.finance.yahoo.com/v10/finance/quoteSummary/{ticker}'\
    + '?modules=summaryProfile,defaultKeyStatistics&corsDomain=finance.yahoo.com'
url = url.format(query_id=2, ticker=ticker)
print(url)

headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36',
}

r = requests.get(url, headers=headers)
if r.status_code != 200:
    print(r.status_code, ticker)

json_data = r.json()['quoteSummary']['result'][0]
print(json_data)

https://query2.finance.yahoo.com/v10/finance/quoteSummary/AAPL?modules=summaryProfile,defaultKeyStatistics&corsDomain=finance.yahoo.com
{'defaultKeyStatistics': {'maxAge': 1, 'priceHint': {'raw': 2, 'fmt': '2', 'longFmt': '2'}, 'enterpriseValue': {'raw': 2507813421056, 'fmt': '2.51T', 'longFmt': '2,507,813,421,056'}, 'forwardPE': {'raw': 27.368912, 'fmt': '27.37'}, 'profitMargins': {'raw': 0.23451, 'fmt': '23.45%'}, 'floatShares': {'raw': 16670609616, 'fmt': '16.67B', 'longFmt': '16,670,609,616'}, 'sharesOutstanding': {'raw': 16687599616, 'fmt': '16.69B', 'longFmt': '16,687,599,616'}, 'sharesShort': {'raw': 90213531, 'fmt': '90.21M', 'longFmt': '90,213,531'}, 'sharesShortPriorMonth': {'raw': 123121920, 'fmt': '123.12M', 'longFmt': '123,121,920'}, 'sharesShortPreviousMonthDate': {'raw': 1622160000, 'fmt': '2021-05-28'}, 'dateShortInterest': {'raw': 1625011200, 'fmt': '2021-06-30'}, 'sharesPercentSharesOut': {'raw': 0.0054, 'fmt': '0.54%'}, 'heldPercentInsiders': {'raw': 0.00066, 'fmt': 

In [52]:
def _parse_raw_values(json_data):
    new_row = {}
    for key in json_data.keys():
        if type(json_data[key]) == dict and 'raw' in json_data[key]:
            new_row[key] = json_data[key]['raw']
            continue
        # if type(json_data[key]) in [list, dict] and len(json_data[key]) == 0:
        #     new_row[key] = None
        #     continue
        new_row[key] = json_data[key]

    return new_row

In [56]:
_parse_raw_values(json_data['summaryProfile'])

{'address1': 'One Apple Park Way',
 'city': 'Cupertino',
 'state': 'CA',
 'zip': '95014',
 'country': 'United States',
 'phone': '408-996-1010',
 'website': 'http://www.apple.com',
 'industry': 'Consumer Electronics',
 'sector': 'Technology',
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. It also sells various related services. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, HomePod, iPod touch, and other Apple-branded and third-party accessories. It also provides AppleCare support services; cloud services store services; and operates various platforms, including the App Store, that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In additi

In [57]:
_parse_raw_values(json_data['defaultKeyStatistics'])

{'maxAge': 1,
 'priceHint': 2,
 'enterpriseValue': 2507813421056,
 'forwardPE': 27.368912,
 'profitMargins': 0.23451,
 'floatShares': 16670609616,
 'sharesOutstanding': 16687599616,
 'sharesShort': 90213531,
 'sharesShortPriorMonth': 123121920,
 'sharesShortPreviousMonthDate': 1622160000,
 'dateShortInterest': 1625011200,
 'sharesPercentSharesOut': 0.0054,
 'heldPercentInsiders': 0.00066,
 'heldPercentInstitutions': 0.58561003,
 'shortRatio': 1.24,
 'shortPercentOfFloat': 0.0054,
 'beta': 1.20729,
 'impliedSharesOutstanding': {},
 'morningStarOverallRating': {},
 'morningStarRiskRating': {},
 'category': None,
 'bookValue': 4.146,
 'priceToBook': 35.250843,
 'annualReportExpenseRatio': {},
 'ytdReturn': {},
 'beta3Year': {},
 'totalAssets': {},
 'yield': {},
 'fundFamily': None,
 'fundInceptionDate': {},
 'legalType': None,
 'threeYearAverageReturn': {},
 'fiveYearAverageReturn': {},
 'priceToSalesTrailing12Months': {},
 'lastFiscalYearEnd': 1601078400,
 'nextFiscalYearEnd': 1664150400

## Quarterly params

In [58]:
DEFAULT_TYPE_LIST = [
    'quarterlyTotalCapitalization',
    'quarterlyTotalRevenue',
    'quarterlyNetIncome',
    'quarterlyFreeCashFlow',
    'quarterlyTotalAssets',
    'quarterlyEBITDA',
    'quarterlyNetDebt',
    'quarterlyGrossProfit',
    'quarterlyWorkingCapital',
    'quarterlyCashAndCashEquivalents',
    'quarterlyResearchAndDevelopment',
    'quarterlyCashDividendsPaid',
]

In [62]:
base_url = ('https://query{query_id}.finance.yahoo.com/ws/fundamentals-timeseries/v1/finance/timeseries'
            '/{ticker}?lang=en-US&region=US&padTimeSeries=false&type={type_str}'
            '&merge=false&period1=493590046&period2={period2}&corsDomain=finance.yahoo.com')
url = base_url.format(query_id=2,
                      ticker=ticker, 
                      type_str=','.join(DEFAULT_TYPE_LIST),
                      period2=int(time.time()))

url

'https://query2.finance.yahoo.com/ws/fundamentals-timeseries/v1/finance/timeseries/AAPL?lang=en-US&region=US&padTimeSeries=false&type=quarterlyTotalCapitalization,quarterlyTotalRevenue,quarterlyNetIncome,quarterlyFreeCashFlow,quarterlyTotalAssets,quarterlyEBITDA,quarterlyNetDebt,quarterlyGrossProfit,quarterlyWorkingCapital,quarterlyCashAndCashEquivalents,quarterlyResearchAndDevelopment,quarterlyCashDividendsPaid&merge=false&period1=493590046&period2=1626816577&corsDomain=finance.yahoo.com'

In [64]:
r = requests.get(url, headers=headers)
if r.status_code != 200:
    print(r.status_code, ticker)

json_data = r.json()
print(json_data)

{'timeseries': {'result': [{'meta': {'symbol': ['AAPL'], 'type': ['quarterlyCashDividendsPaid']}, 'timestamp': [1593475200, 1601424000, 1609372800, 1617148800], 'quarterlyCashDividendsPaid': [{'dataId': 26007, 'asOfDate': '2020-06-30', 'periodType': '3M', 'currencyCode': 'USD', 'reportedValue': {'raw': -3656000000.0, 'fmt': '-3.66B'}}, {'dataId': 26007, 'asOfDate': '2020-09-30', 'periodType': '3M', 'currencyCode': 'USD', 'reportedValue': {'raw': -3511000000.0, 'fmt': '-3.51B'}}, {'dataId': 26007, 'asOfDate': '2020-12-31', 'periodType': '3M', 'currencyCode': 'USD', 'reportedValue': {'raw': -3613000000.0, 'fmt': '-3.61B'}}, {'dataId': 26007, 'asOfDate': '2021-03-31', 'periodType': '3M', 'currencyCode': 'USD', 'reportedValue': {'raw': -3447000000.0, 'fmt': '-3.45B'}}]}, {'meta': {'symbol': ['AAPL'], 'type': ['quarterlyFreeCashFlow']}, 'timestamp': [1593475200, 1601424000, 1609372800, 1617148800], 'quarterlyFreeCashFlow': [{'dataId': 26185, 'asOfDate': '2020-06-30', 'periodType': '3M', 'cu

In [66]:
data = json_data['timeseries']['result'][0]
set(data.keys()).intersection(set(DEFAULT_TYPE_LIST))

{'quarterlyCashDividendsPaid'}

In [67]:
data['quarterlyCashDividendsPaid']

[{'dataId': 26007,
  'asOfDate': '2020-06-30',
  'periodType': '3M',
  'currencyCode': 'USD',
  'reportedValue': {'raw': -3656000000.0, 'fmt': '-3.66B'}},
 {'dataId': 26007,
  'asOfDate': '2020-09-30',
  'periodType': '3M',
  'currencyCode': 'USD',
  'reportedValue': {'raw': -3511000000.0, 'fmt': '-3.51B'}},
 {'dataId': 26007,
  'asOfDate': '2020-12-31',
  'periodType': '3M',
  'currencyCode': 'USD',
  'reportedValue': {'raw': -3613000000.0, 'fmt': '-3.61B'}},
 {'dataId': 26007,
  'asOfDate': '2021-03-31',
  'periodType': '3M',
  'currencyCode': 'USD',
  'reportedValue': {'raw': -3447000000.0, 'fmt': '-3.45B'}}]

In [76]:
def _parse_quarterly_json(json_data: Dict[str, Any]):
    dfs = []
    for data in json_data['timeseries']['result']:
        name_set = set(data.keys()).intersection(set(DEFAULT_TYPE_LIST))
        if len(name_set) == 1:
            name = list(name_set)[0]
            new_data = [
                {'date': row['asOfDate'], name: row['reportedValue']['raw']}
                for row in data[name]
            ]
            dfs.append(pd.DataFrame(new_data))
    if len(dfs) == 0:
        return
    result = reduce(lambda l, r: pd.merge(l, r, on='date', how='left'), dfs)
    for key in set(DEFAULT_TYPE_LIST).difference(set(result.columns)):
        result[key] = None

    result['date'] = result['date'].astype(np.datetime64)
    result = result.sort_values('date', ascending=False)

    return result

In [77]:
_parse_quarterly_json(json_data)

,date,quarterlyCashDividendsPaid,quarterlyFreeCashFlow,quarterlyGrossProfit,quarterlyTotalAssets,quarterlyCashAndCashEquivalents,quarterlyResearchAndDevelopment,quarterlyTotalCapitalization,quarterlyNetDebt,quarterlyWorkingCapital,quarterlyTotalRevenue,quarterlyNetIncome,quarterlyEBITDA
3,2021-03-31,-3.447000e+09,2.171200e+10,3.807900e+10,3.371580e+11,3.846600e+10,5.262000e+09,1.778200e+11,8.317900e+10,1.508000e+10,8.958400e+10,2.363000e+10,None
2,2020-12-31,-3.613000e+09,3.526300e+10,4.432800e+10,3.540540e+11,3.601000e+10,5.163000e+09,1.655050e+11,7.603300e+10,2.159900e+10,1.114390e+11,2.875500e+10,None
1,2020-09-30,-3.511000e+09,1.879200e+10,2.468900e+10,3.238880e+11,3.801600e+10,4.978000e+09,1.640060e+11,7.442000e+10,3.832100e+10,6.469800e+10,1.267300e+10,None
0,2020-06-30,-3.656000e+09,1.470600e+10,2.268000e+10,3.173440e+11,3.338300e+10,4.758000e+09,1.663300e+11,7.934000e+10,4.474700e+10,5.968500e+10,1.125300e+10,None
